<a href="https://colab.research.google.com/github/EdwardFang09/IEE4912/blob/main/Tugas_akhir_(documentation_v2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1> CIT Education Assistant

_________________

Featuers:
- Google search image
- Mentimeter
- Kahoot
- Start class and end class (sqlite (theoretical))
- Web Based CRUD (not done)

Faster whisper
https://github.com/SYSTRAN/faster-whisper

In [ ]:
import torch
print(torch.__version__)
torch.cuda.is_available()

2.6.0+cu126


True

Without trained model

In [ ]:
import speech_recognition as sr
import whisper
import warnings
import time
import webbrowser
import pyttsx3
import os
from faster_whisper import WhisperModel
import sqlite3
import datetime
from thefuzz import fuzz  # Import thefuzz
import numpy as np

# Suppress warnings
warnings.filterwarnings("ignore")

# Initialize speech recognition, TTS engine, and Whisper model
recognizer = sr.Recognizer()
engine = pyttsx3.init()

model_size = "turbo"  # or smaller, depending on your resources
base_model = WhisperModel(model_size, device="cuda", compute_type="float32") # or cpu

assistant_name = "sora mk.0"
should_run = True
source = sr.Microphone()

# Database setup (SQLite)
DATABASE_FILE = "class_data.db"  # Name of your database file
conn = sqlite3.connect(DATABASE_FILE)
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS classes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        start_time TEXT,
        end_time TEXT
    )
''')
conn.commit()


def respond(text):
    engine.say(text)
    engine.runAndWait()

def listen_for_command():
    with source as s:
        print("Listening for commands...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        with open("command.wav", "wb") as f:
            f.write(audio.get_wav_data())

        command, info = base_model.transcribe(r"command.wav", language='en')
        print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

        if command:
            full_command_text = ""
            for segment in command:
                print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
                full_command_text += segment.text + " "
            full_command_text = full_command_text.strip().lower()

            # Check for "sora" first
            if "sora" in full_command_text:
                print("Sora detected. Proceeding with command recognition.")
                # Proceed to command matching
            else:
                print("No 'sora' detected. Waiting for trigger word.")
                return None  # Wait for the trigger word

            # Fuzzy matching logic (now only runs IF "sora" is present)
            best_match = None
            best_score = 0

            commands = {
                "find": "find",
                "open mentimeter": "open mentimeter",
                "open kahoot": "open kahoot",
                "start class": "start class",
                "end class": "end class",
                "exit": "exit",
                "google": "google",
                "chrome":  "chrome",
                "browser": "browser"

            }

            scores = []
            for cmd, target in commands.items():
                score = fuzz.ratio(full_command_text, target)
                scores.append(score)

                if score > best_score:
                    best_score = score
                    best_match = cmd

            # One Sigma Calculation and Threshold
            scores_array = np.array(scores)
            average_score = np.mean(scores_array)
            standard_deviation = np.std(scores_array)

            confidence_threshold = average_score + standard_deviation

            print(f"Average Score: {average_score}")
            print(f"Standard Deviation: {standard_deviation}")
            print(f"Confidence Threshold: {confidence_threshold}")


            if best_score >= confidence_threshold:
                print("Matched command:", best_match)
                return best_match
            else:
                print("Command not recognized with sufficient confidence.")
                return None

        return None

    except sr.UnknownValueError:
        print("Could not understand audio. Please try again.")
        return None
    except sr.RequestError:
        print("Unable to access the speech recognition API.")
        return None



def perform_command(command):
    global should_run
    if command:
        print("Command: ", command)
        if "find" in command:
            query = command.replace("find", "").strip()
            respond(f"Finding images of {query}")
            try:
                search_url = f"https://www.google.com/search?q={query}&tbm=isch"
                webbrowser.open(search_url)
            except Exception as e:
                respond(f"Sorry, I encountered an error during image search: {e}")
        elif "open mentimeter" in command:
            respond("Opening Mentimeter.")
            webbrowser.open("https://www.mentimeter.com/")
        elif "open kahoot" in command:
            respond("Opening Kahoot.")
            webbrowser.open("https://kahoot.com/")
        elif "start class" in command:
            start_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute("INSERT INTO classes (start_time) VALUES (?)", (start_time,))  # Only start time
            conn.commit()
            respond("Class started.")
        elif "end class" in command:
            end_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute("UPDATE classes SET end_time =? WHERE end_time IS NULL ORDER BY id DESC LIMIT 1", (end_time,)) # Update the latest started class
            conn.commit()
            respond("Class ended.")
        elif "google" in command or "chrome" in command or "browser" in command:
            respond("Opening Google.")
            webbrowser.open("www.google.com")  # Or "https://www.google.com"
        elif "exit" in command:
            should_run = False
        else:
            respond("I don't understand that command.")


def main():
    while should_run:
        command = listen_for_command()  # Directly listen for command
        perform_command(command)
        time.sleep(1)
    respond("Goodbye.")
    conn.close() # Close database connection when exiting

if __name__ == "__main__":
    main()

Listening for commands...
Detected language 'en' with probability 1.000000
[0.00s -> 2.36s]  Okay, so I'll open Google.
No 'sora' detected. Waiting for trigger word.
Listening for commands...
Detected language 'en' with probability 1.000000
[0.00s -> 2.00s]  Hey Sora Open Google
Sora detected. Proceeding with command recognition.
Average Score: 27.555555555555557
Standard Deviation: 12.893677271447697
Confidence Threshold: 40.44923282700326
Matched command: google
Command:  google
Listening for commands...
Detected language 'en' with probability 1.000000
[0.00s -> 2.20s]  Sora, buka Google.
Sora detected. Proceeding with command recognition.
Average Score: 23.333333333333332
Standard Deviation: 14.514360704718161
Confidence Threshold: 37.84769403805149
Matched command: google
Command:  google
Listening for commands...
Detected language 'en' with probability 1.000000
[0.00s -> 2.04s]  Surah, buka mentimeter.
No 'sora' detected. Waiting for trigger word.
Listening for commands...
Detecte

KeyboardInterrupt: 

Trained model

In [ ]:
# prompt: same like the code above, but i want to use EdwardFang09/whisper-base-TA-2025 from huggingface. reprint the code above with different model i mentioned
import torch
import speech_recognition as sr
import whisper
import warnings
import time
import webbrowser
import pyttsx3
import os
from faster_whisper import WhisperModel
import sqlite3
import datetime
from thefuzz import fuzz  # Import thefuzz
import numpy as np
from transformers import pipeline

# <h1> Sora assistant
# Featuers:
# - Google search image
# - Mentimeter
# - Kahoot
# - Start class and end class (sqlite (theoretical))
# - Web Based CRUD (not done)
# Faster whisper
# https://github.com/SYSTRAN/faster-whisper
print(torch.__version__)
torch.cuda.is_available()
# Without trained model

# Suppress warnings
warnings.filterwarnings("ignore")

# Initialize speech recognition, TTS engine, and Whisper model
recognizer = sr.Recognizer()
engine = pyttsx3.init()

# Load the model from Hugging Face
pipe = pipeline(model="EdwardFang09/whisper-base-TA-2025_v2")


assistant_name = "sora mk.0"
should_run = True
source = sr.Microphone()

# Database setup (SQLite)
DATABASE_FILE = "class_data.db"  # Name of your database file
conn = sqlite3.connect(DATABASE_FILE)
cursor = conn.cursor()

# Create table if it doesn't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS classes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        start_time TEXT,
        end_time TEXT
    )
''')
conn.commit()


def respond(text):
    engine.say(text)
    engine.runAndWait()

def listen_for_command():
    with source as s:
        print("Listening for commands...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        with open("command.wav", "wb") as f:
            f.write(audio.get_wav_data())

        # Use the Hugging Face pipeline for transcription
        transcription = pipe("command.wav")
        full_command_text = transcription["text"].strip().lower()
        print(f"Transcription: {full_command_text}")

        # Check for "sora" first
        if "sora" in full_command_text:
            print("Sora detected. Proceeding with command recognition.")
            # Proceed to command matching
        else:
            print("No 'sora' detected. Waiting for trigger word.")
            return None  # Wait for the trigger word

        # Fuzzy matching logic (now only runs IF "sora" is present)
        best_match = None
        best_score = 0

        commands = {
            "find": "find",
            "open mentimeter": "open mentimeter",
            "open kahoot": "open kahoot",
            "start class": "start class",
            "end class": "end class",
            "exit": "exit",
            "google": "google",
            "chrome": "chrome",
            "browser": "browser"
        }

        scores = []
        for cmd, target in commands.items():
            score = fuzz.ratio(full_command_text, target)
            scores.append(score)

            if score > best_score:
                best_score = score
                best_match = cmd

        # One Sigma Calculation and Threshold
        scores_array = np.array(scores)
        average_score = np.mean(scores_array)
        standard_deviation = np.std(scores_array)

        confidence_threshold = average_score + standard_deviation

        print(f"Average Score: {average_score}")
        print(f"Standard Deviation: {standard_deviation}")
        print(f"Confidence Threshold: {confidence_threshold}")

        if best_score >= confidence_threshold:
            print("Matched command:", best_match)
            return best_match
        else:
            print("Command not recognized with sufficient confidence.")
            return None

    except sr.UnknownValueError:
        print("Could not understand audio. Please try again.")
        return None
    except sr.RequestError:
        print("Unable to access the speech recognition API.")
        return None


def perform_command(command):
    global should_run
    if command:
        print("Command: ", command)
        if "find" in command:
            query = command.replace("find", "").strip()
            respond(f"Finding images of {query}")
            try:
                search_url = f"https://www.google.com/search?q={query}&tbm=isch"
                webbrowser.open(search_url)
            except Exception as e:
                respond(f"Sorry, I encountered an error during image search: {e}")
        elif "mentimeter" in command:
            respond("Opening Mentimeter.")
            webbrowser.open("https://www.mentimeter.com/")
        elif "kahoot" in command:
            respond("Opening Kahoot.")
            webbrowser.open("https://kahoot.com/")
        elif "start class" in command:
            start_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute("INSERT INTO classes (start_time) VALUES (?)", (start_time,))  # Only start time
            conn.commit()
            respond("Class started.")
        elif "end class" in command:
            end_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Get the ID of the latest class with NULL end_time
            cursor.execute("SELECT id FROM classes WHERE end_time IS NULL ORDER BY id DESC LIMIT 1")
            result = cursor.fetchone()

            if result:
                class_id = result[0]
                cursor.execute("UPDATE classes SET end_time = ? WHERE id = ?", (end_time, class_id))
                conn.commit()
                respond("Class ended.")
            else:
                respond("No active class to end.") # Handle case where there is no active class
        elif "google" in command or "chrome" in command or "browser" in command:
            respond("Opening Google.")
            webbrowser.open("www.google.com")  # Or "https://www.google.com"
        elif "exit" in command:
            should_run = False
        else:
            respond("I don't understand that command.")


def main():
    while should_run:
        command = listen_for_command()  # Directly listen for command
        perform_command(command)
        time.sleep(1)
    respond("Goodbye.")
    conn.close() # Close database connection when exiting

if __name__ == "__main__":
    main()


2.6.0+cu126


Device set to use cuda:0


Listening for commands...
Transcription: sora start class
Sora detected. Proceeding with command recognition.
Average Score: 27.555555555555557
Standard Deviation: 23.065660436198755
Confidence Threshold: 50.62121599175431
Matched command: start class
Command:  start class
Listening for commands...
Transcription: sora end class
Sora detected. Proceeding with command recognition.
Average Score: 33.55555555555556
Standard Deviation: 21.592751412922045
Confidence Threshold: 55.1483069684776
Matched command: end class
Command:  end class
Listening for commands...
Transcription: sora end class
Sora detected. Proceeding with command recognition.
Average Score: 33.55555555555556
Standard Deviation: 21.592751412922045
Confidence Threshold: 55.1483069684776
Matched command: end class
Command:  end class
Listening for commands...
Transcription: s
No 'sora' detected. Waiting for trigger word.
Listening for commands...
Transcription: s
No 'sora' detected. Waiting for trigger word.
Listening for co

KeyboardInterrupt: 

In [ ]:
# prompt: same like the code above, but i want to use EdwardFang09/whisper-base-TA-2025 from huggingface. reprint the code above with different model i mentioned
import torch
import speech_recognition as sr
import whisper
import warnings
import time
import webbrowser
import pyttsx3
import os
from faster_whisper import WhisperModel
import sqlite3
import datetime
from thefuzz import fuzz  # Import thefuzz
import numpy as np
from transformers import pipeline

# <h1> Sora assistant
# Featuers:
# - Google search image
# - Mentimeter
# - Kahoot
# - Start class and end class (sqlite (theoretical))
# - Web Based CRUD (not done)
# Faster whisper
# https://github.com/SYSTRAN/faster-whisper
print(torch.__version__)
torch.cuda.is_available()
# Without trained model

# Suppress warnings
warnings.filterwarnings("ignore")

# Initialize speech recognition, TTS engine, and Whisper model
recognizer = sr.Recognizer()
engine = pyttsx3.init()

# Load the model from Hugging Face
pipe = pipeline(model="EdwardFang09/whisper-base-TA-2025_v2")


assistant_name = "sora mk.0"
should_run = True
source = sr.Microphone()

# Database setup (SQLite)
DATABASE_FILE = "class_data.db"  # Name of your database file
conn = sqlite3.connect(DATABASE_FILE)
cursor = conn.cursor()

# Create table if it doesn't exist
# Create tables if they don't exist
cursor.execute('''
    CREATE TABLE IF NOT EXISTS user (
        user_id INTEGER PRIMARY KEY AUTOINCREMENT,
        username TEXT,
        password TEXT
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS log (
        task_id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER,
        task_name TEXT,
        created_at DATETIME,
        FOREIGN KEY (user_id) REFERENCES user(user_id)
    )
''')

cursor.execute('''
    CREATE TABLE IF NOT EXISTS classes (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        start_time DATETIME,
        end_time DATETIME
    )
''')
conn.commit()

def respond(text):
    engine.say(text)
    engine.runAndWait()

def listen_for_command():
    with source as s:
        print("Listening for commands...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        with open("command.wav", "wb") as f:
            f.write(audio.get_wav_data())

        # Use the Hugging Face pipeline for transcription
        transcription = pipe("command.wav")
        full_command_text = transcription["text"].strip().lower()
        print(f"Transcription: {full_command_text}")

        # Check for "sora" first
        if "sora" in full_command_text:
            print("Sora detected. Proceeding with command recognition.")
            # Proceed to command matching
        else:
            print("No 'sora' detected. Waiting for trigger word.")
            return None  # Wait for the trigger word

        # Fuzzy matching logic (now only runs IF "sora" is present)
        best_match = None
        best_score = 0

        commands = {
            "find": "find",
            "open mentimeter": "open mentimeter",
            "open kahoot": "open kahoot",
            "start class": "start class",
            "end class": "end class",
            "exit": "exit",
            "google": "google",
            "chrome": "chrome",
            "browser": "browser"
        }

        scores = []
        for cmd, target in commands.items():
            score = fuzz.ratio(full_command_text, target)
            scores.append(score)

            if score > best_score:
                best_score = score
                best_match = cmd

        # One Sigma Calculation and Threshold
        scores_array = np.array(scores)
        average_score = np.mean(scores_array)
        standard_deviation = np.std(scores_array)

        confidence_threshold = average_score + standard_deviation

        print(f"Average Score: {average_score}")
        print(f"Standard Deviation: {standard_deviation}")
        print(f"Confidence Threshold: {confidence_threshold}")

        if best_score >= confidence_threshold:
            print("Matched command:", best_match)
            return best_match
        else:
            print("Command not recognized with sufficient confidence.")
            return None

    except sr.UnknownValueError:
        print("Could not understand audio. Please try again.")
        return None
    except sr.RequestError:
        print("Unable to access the speech recognition API.")
        return None


def perform_command(command):
    global should_run
    if command:
        print("Command: ", command)
        if "find" in command:
            query = command.replace("find", "").strip()
            respond(f"Finding images of {query}")
            try:
                search_url = f"https://www.google.com/search?q={query}&tbm=isch"
                webbrowser.open(search_url)
            except Exception as e:
                respond(f"Sorry, I encountered an error during image search: {e}")
        elif "mentimeter" in command:
            respond("Opening Mentimeter.")
            webbrowser.open("https://www.mentimeter.com/")
        elif "kahoot" in command:
            respond("Opening Kahoot.")
            webbrowser.open("https://kahoot.com/")
        elif "start class" in command:
            start_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            cursor.execute("INSERT INTO classes (start_time) VALUES (?)", (start_time,))  # Only start time
            conn.commit()
            respond("Class started.")
        elif "end class" in command:
            end_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            # Get the ID of the latest class with NULL end_time
            cursor.execute("SELECT id FROM classes WHERE end_time IS NULL ORDER BY id DESC LIMIT 1")
            result = cursor.fetchone()

            if result:
                class_id = result[0]
                cursor.execute("UPDATE classes SET end_time = ? WHERE id = ?", (end_time, class_id))
                conn.commit()
                respond("Class ended.")
            else:
                respond("No active class to end.") # Handle case where there is no active class
        elif "google" in command or "chrome" in command or "browser" in command:
            respond("Opening Google.")
            webbrowser.open("www.google.com")  # Or "https://www.google.com"
        elif "exit" in command:
            should_run = False
        else:
            respond("I don't understand that command.")


def main():
    while should_run:
        command = listen_for_command()  # Directly listen for command
        perform_command(command)
        time.sleep(1)
    respond("Goodbye.")
    conn.close() # Close database connection when exiting

if __name__ == "__main__":
    main()
